In [255]:
import pandas as pd
import datetime as dt
import regex as re
import os
from pathlib import Path
import utils_laramie
import numpy as np

### Read in the data

In [205]:
def get_all_raw_data(dirpath = Path('raw_data/')):
    csv_list = [filename for filename in os.listdir(dirpath)]
    return csv_list

In [206]:
def clean_col_names(list_dfs, list_csvs):

    list_cols = [df.columns.to_list() for df in list_dfs]
    zipped = zip(list_csvs,list_cols)
    name_list = []
    column_list = []
    for tup in list(zipped):
        name = tup[0]
        match = re.match(r'(\w+)_',name)
        name = match.group(1)
        for col in tup[1]:
            if name.lower() != col.lower():
                name_list.append((name.upper()+'_'+col.lower()))
            else:
                name_list.append(name.upper())

    return name_list


In [207]:
def drop_unnamed(df):
    for col in df.columns:
        if 'Unnamed' in str(col):
            df.drop(columns= col, inplace=True)
    return df
    

In [208]:
def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
    """concat a list of csvs into a single df"""
    # make a list of dataframes
    list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
    for df in list_dfs:
        df.drop_duplicates(inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['date'] = pd.to_datetime(df['date'])
        df = drop_unnamed(df)
    clean_headers = clean_col_names(list_dfs,list_of_csvs)
    merged_df = pd.concat(list_dfs, axis=1, join= 'inner')
    merged_df.columns = clean_headers
    return merged_df
x = get_df()
x 

,SPY_date,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR_date,TR,VIX_date,VIX_open,VIX_high,VIX_low,VIX_close
0,2015-12-01,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,2015-12-02,2.7700,1990-01-02,17.24,17.24,17.24,17.24
1,2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2015-12-03,4.3989,1990-01-03,18.19,18.19,18.19,18.19
2,2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,2015-12-04,4.3900,1990-01-04,19.22,19.22,19.22,19.22
3,2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,2015-12-07,2.5295,1990-01-05,20.11,20.11,20.11,20.11
4,2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2015-12-08,2.5090,1990-01-08,20.26,20.26,20.26,20.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,2022-03-24,6.7000,1996-04-15,17.54,17.54,16.82,16.83
1589,2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,2022-03-25,4.5500,1996-04-16,16.51,16.56,16.21,16.45
1590,2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,2022-03-28,5.8500,1996-04-17,16.50,17.30,16.36,17.07
1591,2022-03-28,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974,2022-03-29,6.2300,1996-04-18,16.69,17.06,16.24,16.41


In [209]:
def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
    """concat a list of csvs into a single df"""
    list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
    for df in list_dfs:
        df.drop_duplicates(inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['date'] = pd.to_datetime(df['date'])
        df = drop_unnamed(df)
        df.set_index('date',inplace=True)
    clean_headers = clean_col_names(list_dfs,list_of_csvs)
    merged_df = pd.concat(list_dfs, axis=1, join= 'inner')  
    merged_df.columns = clean_headers
    return merged_df
x = get_df()
x 

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close
date,,,,,,,,,,,,
2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91
2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11
2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81
2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,15.84
2015-12-08,206.48,208.2890,205.7800,206.99,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,17.60
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,5.8800,23.04,24.03,22.64,23.57
2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,6.7000,23.61,23.79,21.49,21.67
2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,4.5500,21.87,22.86,20.80,20.81


### Create Weekly Range

In [177]:
df = get_df(get_all_raw_data())
df['weekly_range'] = 0


In [178]:
# week_group = df.groupby(by = [df.index.year, df.index.month, df.index.week]).mean()
df.head(3)

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
date,,,,,,,,,,,,,
2015-12-02,210.6,211.00,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91,0
2015-12-03,208.9,209.15,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11,0
2015-12-04,206.1,209.97,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81,0


In [210]:
week_high = 0  
week_low = 999999
for index, row in df.iterrows():
    if df.loc[index, 'DayOfWeek'] == 'Monday':
        week_high = df.loc[index,'SPY_high']
        week_low = df.loc[index,'SPY_low']
    else: 
        if df.loc[index,'SPY_high']>week_high:
            week_high=df.loc[index,'SPY_high']
        if df.loc[index,'SPY_low']<week_low:
            week_low=df.loc[index,'SPY_low']
        if df.loc[index,'DayOfWeek'] == 'Friday':
            df.loc[index,'weekly_range'] = week_high - week_low

### Group by year and week, then split into x (features) and y (target)

In [214]:
week_group = df.groupby(by = [df.index.isocalendar().year, df.index.isocalendar().week,df.index.isocalendar().day]).mean()


SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  weekly_range  
year week day                                    
2015 49   3      14.71      15.91        0.0000  
          4      15.86      18.11        0.0000  
          5      14.69      14.81        6.2489  
     50   1      15.58      15.84        0.0000  
          2      16.52      17.60        0.0000

In [288]:
idx = pd.IndexSlice
week_group.loc[idx[2015, 49]].index

Index([3, 4, 5], dtype='object', name='day')

In [ ]:

df_x = week_group.drop(columns= 'weekly_range')
df_y = pd.DataFrame(week_group['weekly_range'])
week_group.head()


In [215]:
display(df_y.head())
display(df_x.head())

weekly_range
year week day              
2015 49   3          0.0000
          4          0.0000
          5          6.2489
     50   1          0.0000
          2          0.0000

SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  
year week day                      
2015 49   3      14.71      15.91  
          4      15.86      18.11  
          5      14.69      14.81  
     50   1      15.58      15.84  
          2      16.52      17.60

### Testing for get_array method

In [184]:
wk_grp_ind = week_group.index.to_list()
week_numbers = [tup[1] for tup in wk_grp_ind]
year_numbers = set([tup[0] for tup in wk_grp_ind])


In [185]:
year_numbers = list(year_numbers)

In [186]:
year_numbers = sorted(list(year_numbers))

In [187]:
df_x.head()

SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  
year week day                      
2015 49   3      14.71      15.91  
          4      15.86      18.11  
          5      14.69      14.81  
     50   1      15.58      15.84  
          2      16.52      17.60

### get_array method

In [216]:
def get_array(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    week_data = []

    for yr_num in year_numbers:
        for wk_num in week_numbers: 
            try:
                week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
            except:
                continue
            
    return week_data


In [286]:
def get_array_test(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    

    for yr_num in year_numbers:
        for wk_num in week_numbers:
            
                try:
                    num_days = len(week_group.loc[idx[yr_num, wk_num]].index.to_list())
                    if num_days==5:
                        week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
                except:
                    continue
            
    return week_data

### Check for Nans in Target df. Shift so that target matches features

In [278]:
df_y_shifted = df_y[df_y.weekly_range != 0].shift(-1)
df_y_shifted.isna().sum()

weekly_range    1
dtype: int64

In [ ]:
display(df_x.head(20))
display(df_y[df_y.weekly_range != 0].head(7).shift(-1))

In [264]:
x_data = get_array(df_x)
y_data = get_array(df_y_shifted)


In [287]:
x_data_test = get_array_test(df_x)
y_data_test = get_array_test(df_y_shifted)

TypeError: array() missing required argument 'object' (pos 1)

In [285]:
x_data_test

[array([[2.09200000e+02, 2.09729500e+02, 2.07200000e+02, 2.08270000e+02,
         1.02027111e+08, 3.74705000e+05, 2.08276128e+02, 2.52950000e+00,
         1.56500000e+01, 1.71800000e+01, 1.55800000e+01, 1.58400000e+01],
        [2.06480000e+02, 2.08289000e+02, 2.05780000e+02, 2.06990000e+02,
         1.03372367e+08, 3.87782000e+05, 2.06966276e+02, 2.50900000e+00,
         1.76900000e+01, 1.83300000e+01, 1.65200000e+01, 1.76000000e+01],
        [2.06200000e+02, 2.08680000e+02, 2.04180000e+02, 2.05330000e+02,
         1.62401537e+08, 5.86210000e+05, 2.06034646e+02, 4.50000000e+00,
         1.80500000e+01, 2.01300000e+01, 1.57200000e+01, 1.96100000e+01],
        [2.05440000e+02, 2.07430000e+02, 2.05140000e+02, 2.05860000e+02,
         1.16128858e+08, 4.04992000e+05, 2.06102975e+02, 2.29000000e+00,
         1.92500000e+01, 1.97200000e+01, 1.81300000e+01, 1.93400000e+01],
        [2.03380000e+02, 2.04140000e+02, 2.01510000e+02, 2.01880000e+02,
         2.11173305e+08, 6.69924000e+05, 2.0315

In [196]:
week_group = df.groupby(by = [df.index.isocalendar().year, df.index.isocalendar().week,df.index.isocalendar().day])
weekly_ind = week_group.groups
# last_day_of_week = []
# for tup in weekly_ind.keys():
#     if tup[1] == 5:
#         last_day_of_week.append(weekly_ind[tup])

weekly_ind

{(2015, 49, 3): [2015-12-02 00:00:00], (2015, 49, 4): [2015-12-03 00:00:00], (2015, 49, 5): [2015-12-04 00:00:00], (2015, 50, 1): [2015-12-07 00:00:00], (2015, 50, 2): [2015-12-08 00:00:00], (2015, 50, 3): [2015-12-09 00:00:00], (2015, 50, 4): [2015-12-10 00:00:00], (2015, 50, 5): [2015-12-11 00:00:00], (2015, 51, 1): [2015-12-14 00:00:00], (2015, 51, 2): [2015-12-15 00:00:00], (2015, 51, 3): [2015-12-16 00:00:00], (2015, 51, 4): [2015-12-17 00:00:00], (2015, 51, 5): [2015-12-18 00:00:00], (2015, 52, 1): [2015-12-21 00:00:00], (2015, 52, 2): [2015-12-22 00:00:00], (2015, 52, 3): [2015-12-23 00:00:00], (2015, 52, 4): [2015-12-24 00:00:00], (2015, 53, 1): [2015-12-28 00:00:00], (2015, 53, 2): [2015-12-29 00:00:00], (2015, 53, 3): [2015-12-30 00:00:00], (2015, 53, 4): [2015-12-31 00:00:00], (2016, 1, 1): [2016-01-04 00:00:00], (2016, 1, 2): [2016-01-05 00:00:00], (2016, 1, 3): [2016-01-06 00:00:00], (2016, 1, 4): [2016-01-07 00:00:00], (2016, 1, 5): [2016-01-08 00:00:00], (2016, 2, 1): [2

In [197]:
def flatten(list_of_lists):
        return [item for sublist in list_of_lists for item in sublist]
last_day_of_week = flatten(last_day_of_week)

In [198]:
last_day_of_week = pd.Series(last_day_of_week)

In [199]:
last_day_of_week

0     2016-01-08
1     2017-01-06
2     2018-01-05
3     2019-01-04
4     2020-01-03
         ...    
313   2016-12-30
314   2017-12-29
315   2018-12-28
316   2019-12-27
317   2021-12-31
Length: 318, dtype: datetime64[ns]

In [200]:
df['DayOfWeek'] = df.index.day_name()

In [201]:
df

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,DayOfWeek
date,,,,,,,,,,,,,,
2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91,0,Wednesday
2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11,0,Thursday
2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81,0,Friday
2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,15.84,0,Monday
2015-12-08,206.48,208.2890,205.7800,206.99,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,17.60,0,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,5.8800,23.04,24.03,22.64,23.57,0,Wednesday
2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,6.7000,23.61,23.79,21.49,21.67,0,Thursday
2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,4.5500,21.87,22.86,20.80,20.81,0,Friday


In [202]:
# week_high = 0  
# week_low = 999999
# for index, row in df.iterrows():
#     if df.loc[index, 'DayOfWeek'] == 'Monday':
#         week_high = df.loc[index,'SPY_high']
#         week_low = df.loc[index,'SPY_low']
#     else: 
#         if df.loc[index,'SPY_high']>week_high:
#             week_high=df.loc[index,'SPY_high']
#         if df.loc[index,'SPY_low']<week_low:
#             week_low=df.loc[index,'SPY_low']
#         if df.loc[index,'DayOfWeek'] == 'Friday':
#             df.loc[index,'weekly_range'] = week_high - week_low
    

In [203]:
display(df.head(15))
display(df.tail(15))

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,DayOfWeek
date,,,,,,,,,,,,,,
2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91,0.0000,Wednesday
2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11,0.0000,Thursday
2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81,6.2489,Friday
2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000,Monday
2015-12-08,206.48,208.2890,205.7800,206.99,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000,Tuesday
2015-12-09,206.20,208.6800,204.1800,205.33,162401537,586210,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000,Wednesday
2015-12-10,205.44,207.4300,205.1400,205.86,116128858,404992,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000,Thursday
2015-12-11,203.38,204.1400,201.5100,201.88,211173305,669924,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195,Friday
2015-12-14,202.10,203.0500,199.9500,203.00,182357840,726916,201.724387,3.1000,24.70,26.81,21.47,22.73,0.0000,Monday


,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,DayOfWeek
date,,,,,,,,,,,,,,
2022-03-09,425.16,429.51,422.83,427.33,110176608,1079760,426.138376,13.26,33.74,34.12,31.39,32.45,0.0000,Wednesday
2022-03-10,422.42,426.43,420.44,425.48,91933914,891241,423.871044,6.89,33.03,34.03,30.23,30.23,0.0000,Thursday
2022-03-11,428.20,428.77,419.53,420.07,90803923,809145,424.040193,9.24,30.43,31.04,28.84,30.75,17.1818,Friday
2022-03-14,420.98,424.55,415.79,417.00,91251505,858504,419.220077,8.76,31.03,33.18,30.06,31.77,0.0000,Monday
2022-03-15,419.66,426.84,418.42,426.17,104219651,920659,422.752423,9.84,33.13,33.83,29.57,29.83,0.0000,Tuesday
2022-03-16,429.94,435.68,424.80,435.55,138130298,1344164,431.339744,10.88,29.02,29.80,26.29,26.67,0.0000,Wednesday
2022-03-17,433.70,441.07,433.19,441.07,100157174,784018,437.706981,7.88,26.51,27.47,25.25,25.67,0.0000,Thursday
2022-03-18,437.81,444.86,437.22,444.31,102327793,790235,441.458916,7.64,26.36,26.82,23.85,23.87,29.0700,Friday
2022-03-21,444.45,446.46,440.68,444.30,85377983,912432,443.796432,5.78,25.14,25.36,22.99,23.53,0.0000,Monday
